In [1]:
import pickle
import numpy as np

In [2]:
# load the pkl
with open("pca_features.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features_pca = pickle.load(file)

In [3]:
print(len(track_ids))
print(features_pca.shape)

11883
(237660, 154)


In [4]:
track_ids

['7pse475uICmWRY5hEkvPvI',
 '6AoYWCzsDls6c61Zp5qTuU',
 '6QZtmgaUkxWk3Rlq3Upll6',
 '1jBmW1fRXgLhy1XwE0SJTN',
 '4GYWNXDrG5wWOJteicSy7b',
 '0r0UDwngm74U5ccMQ9gMBM',
 '3Ot04s80MZBhiYP2wHo0is',
 '3EYTVekyY04eqdoi2iGlgJ',
 '4NQQ5fK6qOEMsJNlhjUkIg',
 '2dLWb2s1QtgpzAGOaBVa0h',
 '4UipF3IyxsyStTstiwGHhV',
 '1mnqraQ8oV8MX92rdOFLWW',
 '4QtiVmuA88tPQiCOHZuQ5b',
 '4dGLMiOBMUD74iiAGQ17cK',
 '41SwdQIX8Hy2u6fuEDgvWr',
 '6Ti0coAmnSsv3Wx09RhoY6',
 '78L2RUFISfqkmZMTCLYQJC',
 '28XvT0F0E4sVamI6JRVnmU',
 '2j7mcsEz4n5XxMMqqjjB6p',
 '5fIq2ovEhDIyV5Hx1epC4c',
 '5o7ZDvfOry1MNhNLBOJulo',
 '3HYXqOAm2Q8b2OyYqzv8d4',
 '4ESkG5iVSKtUANJK0NPz1w',
 '7KbF6AdprOXEEHlsq11Z6d',
 '6JVn2NEZ3JQLaHX5qG2Yho',
 '6k5qYxiFt76meFmKRDtZEz',
 '5O3nMjk8xL7n4TIh0tym4i',
 '50oMV9ChVZchW4zAcsKigA',
 '5AtVmp8JU2aar7PUKx6pdh',
 '7jHJ2MzIKO1kIMxGamKRKn',
 '15DwFznkBJir7AK9PyMyRR',
 '5UVBumEwdUnzvqxrXOYLFA',
 '0qgrrDnUUhyxpxbBznUnzg',
 '13aKGvTil3UUw1yk0pA9fG',
 '1HlGblweLq0eAwnb0NY1EQ',
 '6Nx1opJm0kavd1WWMpnlho',
 '0Mft55GDMuD2rFnrSeFn3U',
 

In [5]:
features_pca_1000=features_pca
track_ids_1000=track_ids

In [6]:
features_pca_1000[:20].shape

(20, 154)

In [7]:

num_canciones = features_pca_1000.shape[0] // 20 # como a cada cancion le pertenece 20 arrays aqui se pone en 1 por cada cancion
features_pca_20en1=features_pca_1000.reshape(num_canciones, 20 * features_pca.shape[1])
print(features_pca_20en1.shape)


(11883, 3080)


In [28]:
import numpy as np
import heapq
from collections import Counter

def knn_lineal_pca(cancionesc,track_ids,consultas, top_k):
    all_neighbors=[]
    
    for consulta in consultas:
        nearest_neighbors = [(-float('inf'), -1)] * 20#20 caracteristicas
        for i, embedding in enumerate(cancionesc):
            dist = -np.linalg.norm(consulta - embedding)#consigo mi distancia euclidiana invertida para usar como max-heap
            if dist > nearest_neighbors[0][0]:
                heapq.heappop(nearest_neighbors)
                heapq.heappush(nearest_neighbors, (dist, i))
        nearest_neighbors = [i for dit, i in nearest_neighbors]
        all_neighbors.extend(nearest_neighbors)

    song_indices = [neighbor // 20 for neighbor in all_neighbors]
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices]

    song_counts = Counter(track_id_votes)
    top_songs = song_counts.most_common(top_k)  

    return top_songs


indicecancion = 0 
query = features_pca[indicecancion * 20 : (indicecancion + 1) * 20]  # consulta
top_k = 3  #cuantos k vecinos cercanos quiero

resultado = knn_lineal_pca(features_pca, track_ids, query, top_k)

# Mostrar resultados
for i, (song_id, votes) in enumerate(resultado, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")


Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8


busqueda por rango

In [115]:
import heapq

def knn_busqueda_rango(query, cancionesc, rado):
    canciones_en_radio = []
    
    for i, embedding in enumerate(cancionesc):
        dist = np.linalg.norm(query - embedding)  
        if dist <= radio: 
            heapq.heappush(canciones_en_radio, (dist, i))  
    
    return canciones_en_radio

query = features_pca_20en1[indicecancion]  
radio = 4  # Radio de búsqueda

resultados = knn_busqueda_rango(query, features_pca_20en1, radio)

print("Número de Canciones dentro del radio " + str(radio) + " : " + str(len(resultados)))

while resultados:
    dist, idx = heapq.heappop(resultados)
    print("Indice de la canción : "+str(idx)+" ;Distancia: "+str(dist))


Número de Canciones dentro del radio 4 : 6
Indice de la canción : 15 ;Distancia: 0.0
Indice de la canción : 96 ;Distancia: 3.8466039
Indice de la canción : 14 ;Distancia: 3.884486
Indice de la canción : 472 ;Distancia: 3.899892
Indice de la canción : 569 ;Distancia: 3.9528081
Indice de la canción : 905 ;Distancia: 3.9970117


In [116]:
query = features_pca_20en1[indicecancion]  
radio = 4.2  # Radio de búsqueda

resultados = knn_busqueda_rango(query, features_pca_20en1, radio)

print("Número de Canciones dentro del radio " + str(radio) + " : " + str(len(resultados)))

while resultados:
    dist, idx = heapq.heappop(resultados)
    print("Indice de la canción : "+str(idx)+" ;Distancia: "+str(dist))

Número de Canciones dentro del radio 4.2 : 19
Indice de la canción : 15 ;Distancia: 0.0
Indice de la canción : 96 ;Distancia: 3.8466039
Indice de la canción : 14 ;Distancia: 3.884486
Indice de la canción : 472 ;Distancia: 3.899892
Indice de la canción : 569 ;Distancia: 3.9528081
Indice de la canción : 905 ;Distancia: 3.9970117
Indice de la canción : 996 ;Distancia: 4.0165
Indice de la canción : 394 ;Distancia: 4.038455
Indice de la canción : 59 ;Distancia: 4.0740986
Indice de la canción : 73 ;Distancia: 4.088399
Indice de la canción : 209 ;Distancia: 4.0952306
Indice de la canción : 204 ;Distancia: 4.0996323
Indice de la canción : 524 ;Distancia: 4.1125727
Indice de la canción : 697 ;Distancia: 4.1315236
Indice de la canción : 72 ;Distancia: 4.1464515
Indice de la canción : 266 ;Distancia: 4.1757545
Indice de la canción : 230 ;Distancia: 4.1782913
Indice de la canción : 730 ;Distancia: 4.184679
Indice de la canción : 406 ;Distancia: 4.1887183


In [117]:
query = features_pca_20en1[indicecancion]  
radio = 4.5  # Radio de búsqueda

resultados = knn_busqueda_rango(query, features_pca_20en1, radio)

print("Número de Canciones dentro del radio " + str(radio) + " : " + str(len(resultados)))

while resultados:
    dist, idx = heapq.heappop(resultados)
    print("Indice de la canción : "+str(idx)+" ;Distancia: "+str(dist))

Número de Canciones dentro del radio 4.5 : 67
Indice de la canción : 15 ;Distancia: 0.0
Indice de la canción : 96 ;Distancia: 3.8466039
Indice de la canción : 14 ;Distancia: 3.884486
Indice de la canción : 472 ;Distancia: 3.899892
Indice de la canción : 569 ;Distancia: 3.9528081
Indice de la canción : 905 ;Distancia: 3.9970117
Indice de la canción : 996 ;Distancia: 4.0165
Indice de la canción : 394 ;Distancia: 4.038455
Indice de la canción : 59 ;Distancia: 4.0740986
Indice de la canción : 73 ;Distancia: 4.088399
Indice de la canción : 209 ;Distancia: 4.0952306
Indice de la canción : 204 ;Distancia: 4.0996323
Indice de la canción : 524 ;Distancia: 4.1125727
Indice de la canción : 697 ;Distancia: 4.1315236
Indice de la canción : 72 ;Distancia: 4.1464515
Indice de la canción : 266 ;Distancia: 4.1757545
Indice de la canción : 230 ;Distancia: 4.1782913
Indice de la canción : 730 ;Distancia: 4.184679
Indice de la canción : 406 ;Distancia: 4.1887183
Indice de la canción : 137 ;Distancia: 4.21

Testando la similitud 

In [20]:
import pandas as pd
df = pd.read_csv('spotify_songs.csv',sep=',')

# Mostrar las primeras filas del DataFrame
df

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,...,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
18450,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,-6.224,1,0.1800,0.06730,0.000006,0.2140,0.647,129.990,260240,en
18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,...,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,...,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [ ]:
df.iloc[[13,14,95,471,568]]


In [90]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('spotify_songs.csv', sep=',')

filas_seleccionadas = df.iloc[[14, 15, 96, 472, 569]]

caracteristicas_acusticas = ['danceability', 'energy', 'loudness', 'tempo', 'duration_ms', 'valence']

similaridad = cosine_similarity(filas_seleccionadas[caracteristicas_acusticas])
similaridad_df = pd.DataFrame(similaridad, index=filas_seleccionadas.index, columns=filas_seleccionadas.index)

print(similaridad_df)



     14   15   96   472  569
14   1.0  1.0  1.0  1.0  1.0
15   1.0  1.0  1.0  1.0  1.0
96   1.0  1.0  1.0  1.0  1.0
472  1.0  1.0  1.0  1.0  1.0
569  1.0  1.0  1.0  1.0  1.0
